In [19]:
import pandas as pd
import sqlite3
import csvkit

# Read the CSV files
customers = pd.read_csv('/Users/sasanksasi/Downloads/project/untitled folder/customers.csv')
customer_addresses = pd.read_csv('/Users/sasanksasi/Downloads/project/untitled folder/customer_addresses.csv')
products = pd.read_csv('/Users/sasanksasi/Downloads/project/untitled folder/products.csv')
product_variants = pd.read_csv('/Users/sasanksasi/Downloads/project/untitled folder/product_variants.csv')
product_prices = pd.read_csv('/Users/sasanksasi/Downloads/project/untitled folder/product_pricing.csv')
orders = pd.read_csv('/Users/sasanksasi/Downloads/project/untitled folder/orders.csv')
order_items = pd.read_csv('/Users/sasanksasi/Downloads/project/untitled folder/order_items.csv')

# Create a database connection
# Create a SQLite database
conn = sqlite3.connect('database.db')

# Load the CSV data into the SQLite database
customers.to_sql('customers', conn, if_exists='replace', index=False)
customer_addresses.to_sql('customer_addresses', conn, if_exists='replace', index=False)
products.to_sql('products', conn, if_exists='replace', index=False)
product_variants.to_sql('product_variants', conn, if_exists='replace', index=False)
product_prices.to_sql('product_prices', conn, if_exists='replace', index=False)
orders.to_sql('orders', conn, if_exists='replace', index=False)
order_items.to_sql('order_items', conn, if_exists='replace', index=False)

# Define the SQL queries
query1 = """
    WITH recent_orders AS (
      SELECT o.CustomerID, SUM(oi.Quantity * pp.Price) AS total_amount, COUNT(*) AS order_count
      FROM orders o
      JOIN order_items oi ON o.OrderID = oi.OrderID
      JOIN product_prices pp ON oi.VariantID = pp.VariantID
      WHERE o.OrderDate >= DATE('now', '-6 months')
      GROUP BY o.CustomerID
    )
    SELECT c.Name, AVG(total_amount / order_count) AS avg_order_amount
    FROM recent_orders ro
    JOIN customers c ON ro.CustomerID = c.CustomerID
    GROUP BY c.Name
    ORDER BY avg_order_amount DESC
    LIMIT 5;
"""

# Execute the SQL queries
result1 = pd.read_sql_query(query1, conn)

# Print the results
print("Top 5 customers with highest average order amounts in the last 6 months:")
print(result1)
query = "PRAGMA table_info(orders)"
result = pd.read_sql_query(query, conn)
print(result)

query = "PRAGMA table_info(customers)"
result = pd.read_sql_query(query, conn)
print(result)


Top 5 customers with highest average order amounts in the last 6 months:
                 Name  avg_order_amount
0    Tammy Richardson             254.0
1     Jeffrey Aguilar             253.0
2  Samantha Rodriguez             190.0
3  Crystal Taylor DDS             184.0
4      Kimberly Smith             175.0
   cid        name     type  notnull dflt_value  pk
0    0     OrderID  INTEGER        0       None   0
1    1  CustomerID  INTEGER        0       None   0
2    2   OrderDate     TEXT        0       None   0
   cid        name     type  notnull dflt_value  pk
0    0  CustomerID  INTEGER        0       None   0
1    1        Name     TEXT        0       None   0
2    2       Email     TEXT        0       None   0


In [18]:
print("Customers columns:", customers.columns)
print("Customer Addresses columns:", customer_addresses.columns)
print("Products columns:", products.columns)
print("Product Variants columns:", product_variants.columns)
print("Product Prices columns:", product_prices.columns)
print("Orders columns:", orders.columns)
print("Order Items columns:", order_items.columns)

Customers columns: Index(['CustomerID', 'Name', 'Email'], dtype='object')
Customer Addresses columns: Index(['AddressID', 'CustomerID', 'Address'], dtype='object')
Products columns: Index(['ProductID', 'Name', 'Description'], dtype='object')
Product Variants columns: Index(['VariantID', 'ProductID', 'Attribute1', 'Attribute2'], dtype='object')
Product Prices columns: Index(['PricingID', 'VariantID', 'Price', 'EffectiveDate'], dtype='object')
Orders columns: Index(['OrderID', 'CustomerID', 'OrderDate'], dtype='object')
Order Items columns: Index(['ItemID', 'OrderID', 'VariantID', 'Quantity'], dtype='object')


In [20]:
query2 = """
    WITH this_year_orders AS (
      SELECT o.CustomerID, SUM(oi.Quantity * pp.Price) AS this_year_total
      FROM orders o
      JOIN order_items oi ON o.OrderID = oi.OrderID
      JOIN product_prices pp ON oi.VariantID = pp.VariantID
      WHERE strftime('%Y', o.OrderDate) = strftime('%Y', 'now')
      GROUP BY o.CustomerID
    ),
    last_year_orders AS (
      SELECT o.CustomerID, SUM(oi.Quantity * pp.Price) AS last_year_total
      FROM orders o
      JOIN order_items oi ON o.OrderID = oi.OrderID
      JOIN product_prices pp ON oi.VariantID = pp.VariantID
      WHERE strftime('%Y', o.OrderDate) = strftime('%Y', date('now','-1 year'))
      GROUP BY o.CustomerID
    )
    SELECT c.Name
    FROM this_year_orders ty
    JOIN last_year_orders ly ON ty.CustomerID = ly.CustomerID
    JOIN customers c ON ty.CustomerID = c.CustomerID
    WHERE ty.this_year_total < ly.last_year_total;
"""
result2 = pd.read_sql_query(query2, conn)
print("\nCustomers with lower order value this year compared to previous year:")
print(result2)


Customers with lower order value this year compared to previous year:
                 Name
0  Samantha Rodriguez
1    Tammy Richardson
2       Manuel Graves


In [21]:
customer_id = 1  # Replace with the desired customer ID
query3 = """
    WITH customer_orders AS (
      SELECT pv.ProductID, SUM(oi.Quantity * pp.Price) AS total_amount
      FROM orders o
      JOIN order_items oi ON o.OrderID = oi.OrderID
      JOIN product_variants pv ON oi.VariantID = pv.VariantID
      JOIN product_prices pp ON pv.VariantID = pp.VariantID
      WHERE o.CustomerID = ?
      GROUP BY pv.ProductID
    )
    SELECT p.Name AS product_name, SUM(total_amount) AS cumulative_amount
    FROM customer_orders co
    JOIN products p ON co.ProductID = p.ProductID
    GROUP BY product_name
    ORDER BY cumulative_amount DESC;
"""
result3 = pd.read_sql_query(query3, conn, params=(customer_id,))
print(f"\nCumulative purchase by customer {customer_id}, broken down by product:")
print(result3)


Cumulative purchase by customer 1, broken down by product:
                    product_name  cumulative_amount
0  transition granular paradigms               2584


In [24]:
query4 = """
    WITH product_sales AS (
      SELECT pv.ProductID, pv.VariantID, SUM(oi.Quantity * pp.Price) AS total_amount
      FROM Orders o
      JOIN Order_Items oi ON o.OrderID = oi.OrderID
      JOIN Product_Variants pv ON oi.VariantID = pv.VariantID
      JOIN Product_Prices pp ON pv.VariantID = pp.VariantID
      GROUP BY pv.ProductID, pv.VariantID
    )
    SELECT p.Name AS product_name, pv.Attribute1 || ' ' || pv.Attribute2 AS variant, total_amount
    FROM product_sales ps
    JOIN Products p ON ps.ProductID = p.ProductID
    JOIN Product_Variants pv ON ps.VariantID = pv.VariantID
    ORDER BY total_amount DESC
    LIMIT 5;
"""
result4 = pd.read_sql_query(query4, conn)
print("\nTop 5 selling products, bifurcated by product variants:")
print(result4)


Top 5 selling products, bifurcated by product variants:
                          product_name         variant  total_amount
0  disintermediate visionary solutions     reflect cup          3077
1       optimize end-to-end e-commerce  Democrat woman          2100
2           grow visionary initiatives        safe eye          1845
3      e-enable intuitive partnerships  region section          1781
4      e-enable intuitive partnerships    still during          1512


In [25]:
conn.close()